In [405]:
import re
from collections import Counter, defaultdict, deque
from dataclasses import dataclass, field
from functools import cache
from heapq import heappop, heappush
from itertools import combinations, permutations, product
from math import atan2, inf, pi
from typing import *

import black
import jupyter_black
import networkx as nx
from icecream import ic
from parse import parse

from primefac import primefac

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def first(iterable):
    return next(iter(iterable))

In [30]:
# Day 1: The Tyranny of the Rocket Equation
def fuel_needed(mass):
    fuel = mass // 3 - 2
    if fuel <= 0:
        return 0
    return fuel_needed(fuel) + fuel


modules = [int(x) for x in open("2019/1.txt").read().splitlines()]
print(f"Part 1: {sum(module // 3 - 2 for module in modules)}")
print(f"Part 2: {sum(fuel_needed(module) for module in modules)}")

Part 1: 3295424
Part 2: 4940279


In [95]:
# Day 2: 1202 Program Alarm
def address(pointer):
    return memory[pointer]


def reg(pos):
    return memory[memory[pos]]


def add(ip):
    memory[address(ip + 3)] = reg(ip + 1) + reg(ip + 2)


def mul(ip):
    memory[address(ip + 3)] = reg(ip + 1) * reg(ip + 2)


def run_program():
    ip = 0
    instruction = memory[ip]
    while instruction != 99:
        instructions[instruction][0](ip)
        ip += instructions[instruction][1]
        instruction = memory[ip]


def part1():
    memory[1] = 12
    memory[2] = 2
    run_program()


def part2():
    global memory
    memory = program.copy()
    for noun, verb in product(range(100), range(100)):
        memory[1] = noun
        memory[2] = verb
        run_program()
        if address(0) == 19690720:
            break
        memory = program.copy()
    return noun, verb


# opcode: (instruction, increase of ip)
instructions = {1: (add, 4), 2: (mul, 4)}

program = [int(x) for x in open("2019/2.txt").read().split(",")]
memory = program.copy()
part1()
print(f"Part 1: {address(0)}")  # 5098658
noun, verb = part2()
print(f"Part 2: {100*noun + verb}") # 5064

Part 1: 5098658
Part 2: 5064


In [205]:
# Day 3: Crossed Wires
def create_wire(moves):
    x, y = 0, 0
    cur_step = 1
    wire = {}
    dx = dict(zip("LRUD", [-1, 1, 0, 0]))
    dy = dict(zip("LRUD", [0, 0, 1, -1]))
    for move in moves:
        dir, steps = move
        for step in range(steps):
            x += dx[dir]
            y += dy[dir]
            wire[(x, y)] = cur_step + step
        cur_step += steps
    return wire


def parse_line(line):
    return [tuple((move[0], int(move[1:]))) for move in line.split(",")]


def origo_distance(a):
    return sum(abs(x) for x in a)


a, b = open("2019/3.txt").read().splitlines()
moves_a = parse_line(a)
moves_b = parse_line(b)
# moves_a = parse_line("R75,D30,R83,U83,L12,D49,R71,U7,L72")
# moves_b = parse_line("U62,R66,U55,R34,D71,R55,D58,R83")
a = create_wire(moves_a)
b = create_wire(moves_b)
intersections = {point: a[point] + b[point] for point in a if point in b}
print(f"Part 1: {origo_distance(min(intersections, key=origo_distance))}")  # 2180
print(f"Part 2: {min(intersections.values())}")  # 112316

Part 1: 2180
Part 2: 112316


In [282]:
# Day 4: Secure Container
def no_decrease(number):
    number = str(number)
    return all(b >= a for a, b in zip(number, number[1:]))


def two_consecutive(number):
    number = str(number)
    return any(b == a for a, b in zip(number, number[1:]))


def strip_repeats(number):
    """Returns a string where all numbers that are repeated three times or more are
    removed. E.g. '122233' -> '133'."""
    result = str(number)
    for x in re.findall(r"(.)\1{2,}", str(number)):
        result = result.replace(x, "")
    return result


matches = 0
for number in range(172851, 675869 + 1):
    if no_decrease(number) and two_consecutive(number):
        matches += 1

print(f"Part 1: {matches}")  # 1660

matches = 0
for number in range(172851, 675869 + 1):
    if no_decrease(number) and two_consecutive(number):
        no_long_repeats = strip_repeats(number)
        if no_long_repeats and two_consecutive(int(no_long_repeats)):
            matches += 1
print(f"Part 2: {matches}")  # 1135

Part 1: 1688
Part 2: 1135


In [6]:
# Day 5: Sunny with a change of Asteroids
def parameter(offset, modes):
    return memory[memory[ip + offset]] if modes[-offset] == "0" else memory[ip + offset]


def destination(offset, modes):
    return memory[ip + offset] if modes[-offset] == "0" else ip + offset


def add(modes):
    global ip
    memory[destination(3, modes)] = parameter(1, modes) + parameter(2, modes)
    ip += 4


def mul(modes):
    global ip
    memory[destination(3, modes)] = parameter(1, modes) * parameter(2, modes)
    ip += 4


def poke(modes):
    global ip
    memory[destination(1, modes)] = input_value
    ip += 2


def peek(modes):
    global ip
    mem_address = destination(1, modes)
    ip += 2
    return memory[mem_address]


def jump_if_true(modes):
    global ip
    if parameter(1, modes):
        ip = parameter(2, modes)
    else:
        ip += 3


def jump_if_false(modes):
    global ip
    if not parameter(1, modes):
        ip = parameter(2, modes)
    else:
        ip += 3


def less_than(modes):
    global ip
    if parameter(1, modes) < parameter(2, modes):
        memory[destination(3, modes)] = 1
    else:
        memory[destination(3, modes)] = 0
    ip += 4


def equals(modes):
    global ip
    if parameter(1, modes) == parameter(2, modes):
        memory[destination(3, modes)] = 1
    else:
        memory[destination(3, modes)] = 0
    ip += 4


def run_program(program, input):
    global ip, input_value, memory
    instructions = {
        1: add,
        2: mul,
        3: poke,
        4: peek,
        5: jump_if_true,
        6: jump_if_false,
        7: less_than,
        8: equals,
    }
    input_value = input
    memory = program.copy()
    outputs, ip = [], 0
    opcode = int(str(memory[ip])[-2:])  # Last two digits of instruction field
    while opcode != 99:
        modes = "000" + str(memory[ip])[:-2]  # Everything but last two digits, padded
        result = instructions[opcode](modes)  # Execute instruction
        if result:
            outputs.append(result)
        opcode = int(str(memory[ip])[-2:])  # Last two digits
    return outputs


program = [int(x) for x in open("2019/5.txt").read().split(",")]
print("Part 1:", first(run_program(program, 1)))  # 13294380
print("Part 2:", first(run_program(program, 5)))  # 11460760

Part 1: 13294380
Part 2: 11460760


In [27]:
# Day 6: Universal Orbit Map
lines = open("2019/6.txt").read().splitlines()

G = nx.Graph()
for line in lines:
    u, v = line.split(")")
    G.add_edge(u, v)
orbits = nx.single_target_shortest_path_length(G, "COM")
print(f"Part 1: {sum(dist for _, dist in orbits)}")  # 135690
print(f"Part 2: {nx.shortest_path_length(G, 'YOU', 'SAN') - 2}")  # 298

Part 1: 135690
Part 2: 298


In [4]:
# Day 7: Amplification Circuit
@dataclass
class Amplifier:
    inputs: List[int]
    outputs: List[int]
    memory: List[int]
    ip: int = 0
    running: bool = True


def parameter(offset, modes):
    return memory[memory[ip + offset]] if modes[-offset] == "0" else memory[ip + offset]


def destination(offset, modes):
    return memory[ip + offset] if modes[-offset] == "0" else ip + offset


def add(modes):
    global ip
    memory[destination(3, modes)] = parameter(1, modes) + parameter(2, modes)
    ip += 4


def mul(modes):
    global ip
    memory[destination(3, modes)] = parameter(1, modes) * parameter(2, modes)
    ip += 4


def read_input(modes):
    global ip, inputs
    if inputs:
        memory[destination(1, modes)] = inputs.pop(0)
    else:
        raise RuntimeError("No values in inputs")
    ip += 2


def output(modes):
    global ip
    mem_address = destination(1, modes)
    ip += 2
    return memory[mem_address]


def jump_if_true(modes):
    global ip
    if parameter(1, modes):
        ip = parameter(2, modes)
    else:
        ip += 3


def jump_if_false(modes):
    global ip
    if not parameter(1, modes):
        ip = parameter(2, modes)
    else:
        ip += 3


def less_than(modes):
    global ip
    if parameter(1, modes) < parameter(2, modes):
        memory[destination(3, modes)] = 1
    else:
        memory[destination(3, modes)] = 0
    ip += 4


def equals(modes):
    global ip
    if parameter(1, modes) == parameter(2, modes):
        memory[destination(3, modes)] = 1
    else:
        memory[destination(3, modes)] = 0
    ip += 4


def run_program(amplifier):
    global ip, memory, inputs
    instructions = {
        1: add,
        2: mul,
        3: read_input,
        4: output,
        5: jump_if_true,
        6: jump_if_false,
        7: less_than,
        8: equals,
    }
    memory = amplifier.memory.copy()
    inputs = amplifier.inputs.copy()
    ip = amplifier.ip
    opcode = int(str(memory[ip])[-2:])  # Last two digits of instruction field
    while opcode != 99:
        modes = "000" + str(memory[ip])[:-2]  # Everything but last two digits, padded
        if instructions[opcode] == read_input and not inputs:
            # Waiting for inputs. Store program state and let the next amplifier run
            amplifier.memory = memory.copy()
            amplifier.inputs = inputs.copy()
            amplifier.ip = ip
            return
        result = instructions[opcode](modes)  # Execute instruction
        if instructions[opcode] == output:
            amplifier.outputs.append(result)
        opcode = int(str(memory[ip])[-2:])  # Next opcode
    amplifier.running = False


def run_amplifiers(phase_settings, program, one_round=True):
    amplifiers = [Amplifier([setting], [], program) for setting in phase_settings]
    prev_output = 0  # Input to first amplifier
    while any(amp.running for amp in amplifiers):
        for amplifier in amplifiers:
            amplifier.inputs.append(prev_output)
            run_program(amplifier)
            prev_output = amplifier.outputs.pop(0)
        if one_round:
            return prev_output
    return prev_output


program = [int(x) for x in open("2019/7.txt").read().split(",")]
part1 = max(
    run_amplifiers(phase_settings, program) for phase_settings in permutations(range(5))
)
print(f"Part 1: {part1}")  # 95757
part2 = max(
    run_amplifiers(phase_settings, program, one_round=False)
    for phase_settings in permutations(range(5,10))
)
print(f"Part 2: {part2}")  # 4275738

Part 1: 95757
Part 2: 4275738


In [106]:
# Day 8: Space Image Format
def fewest_zeros(layers):
    "Return index for layer with fewest 0:s"
    min_zeros = 10**6
    index = -1
    for i, layer in enumerate(layers):
        if layer.count(0) < min_zeros:
            min_zeros = layer.count(0)
            index = i
    return index


def pixel_value(layers, position):
    TRANSPARENT = 2
    for layer in range(len(layers)):
        if layers[layer][position] == TRANSPARENT:
            continue
        else:
            return layers[layer][position]
    return TRANSPARENT


def print_image(image):
    for row in range(heigth):
        for col in range(width):
            print("⬛️⬜️"[image[row * width + col] * 2], end="")  # CFLUL
        print()


digits = open("2019/8.txt").read().strip()
width, heigth = 25, 6
digits = [int(x) for x in digits]
pixels_per_layer = width * heigth
num_layers = len(digits) // (pixels_per_layer)

layers = [
    digits[layer * pixels_per_layer : (layer + 1) * pixels_per_layer]
    for layer in range(num_layers)
]

index = fewest_zeros(layers)
part1 = layers[index].count(1) * layers[index].count(2)
print(f"Part 1: {part1}")

image = []
for position in range(pixels_per_layer):
    image.append(pixel_value(layers, position))

print("Part 2:")
print_image(image)

Part 1: 1935
Part 2:
⬛⬜⬜⬛⬛⬜⬜⬜⬜⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛
⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛
⬜⬛⬛⬛⬛⬜⬜⬜⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛
⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛
⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛
⬛⬜⬜⬛⬛⬜⬛⬛⬛⬛⬜⬜⬜⬜⬛⬛⬜⬜⬛⬛⬜⬜⬜⬜⬛


In [4]:
# Day 9: Sensor Boost
@dataclass
class VirtualMachine:
    inputs: List[int]
    outputs: List[int]
    memory: DefaultDict[int, int]
    ip: int = 0
    relative_base: int = 0
    running: bool = True


def address(offset, modes):
    POSITION = "0"
    IMMEDIATE = "1"
    RELATIVE = "2"
    if modes[-offset] == POSITION:
        return memory[ip + offset]
    elif modes[-offset] == IMMEDIATE:
        return ip + offset
    elif modes[-offset] == RELATIVE:
        return memory[ip + offset] + relative_base
    else:
        raise ValueError("Invalid mode", modes[-offset], offset, modes)


def add(modes):
    global ip
    memory[address(3, modes)] = memory[address(1, modes)] + memory[address(2, modes)]
    ip += 4


def mul(modes):
    global ip
    memory[address(3, modes)] = memory[address(1, modes)] * memory[address(2, modes)]
    ip += 4


def read_input(modes):
    global ip, inputs
    if inputs:
        memory[address(1, modes)] = inputs.pop(0)
    else:
        raise RuntimeError("No values in inputs")
    ip += 2


def output(modes):
    global ip
    mem_address = address(1, modes)
    ip += 2
    return memory[mem_address]


def jump_if_true(modes):
    global ip
    if memory[address(1, modes)]:
        ip = memory[address(2, modes)]
    else:
        ip += 3


def jump_if_false(modes):
    global ip
    if not memory[address(1, modes)]:
        ip = memory[address(2, modes)]
    else:
        ip += 3


def less_than(modes):
    global ip
    if memory[address(1, modes)] < memory[address(2, modes)]:
        memory[address(3, modes)] = 1
    else:
        memory[address(3, modes)] = 0
    ip += 4


def equals(modes):
    global ip
    if memory[address(1, modes)] == memory[address(2, modes)]:
        memory[address(3, modes)] = 1
    else:
        memory[address(3, modes)] = 0
    ip += 4


def relative_base_offset(modes):
    global ip, relative_base
    relative_base += memory[address(1, modes)]
    ip += 2


def run_program(vm):
    global ip, memory, inputs, relative_base
    instructions = {
        1: add,
        2: mul,
        3: read_input,
        4: output,
        5: jump_if_true,
        6: jump_if_false,
        7: less_than,
        8: equals,
        9: relative_base_offset,
    }
    memory = vm.memory.copy()
    inputs = vm.inputs.copy()
    ip = vm.ip
    relative_base = vm.relative_base
    opcode = int(str(memory[ip])[-2:])  # Last two digits of instruction field
    while opcode != 99:
        modes = "000" + str(memory[ip])[:-2]  # Everything but last two digits, padded
        if instructions[opcode] == read_input and not inputs:
            # Waiting for inputs. Store program state and let the next amplifier run
            vm.memory = memory.copy()
            vm.inputs = inputs.copy()
            vm.ip = ip
            vm.relative_base = relative_base
            return
        result = instructions[opcode](modes)  # Execute instruction
        if instructions[opcode] == output:
            vm.outputs.append(result)
        opcode = int(str(memory[ip])[-2:])  # Next opcode
    vm.running = False


def run_virtual_machine(input_value=0):
    vm = VirtualMachine([input_value], [], program)
    run_program(vm)
    return vm


def create_program(text):
    program = defaultdict(int)
    for pos, x in enumerate(text.split(",")):
        program[pos] = int(x)
    return program


program = create_program(open("2019/9.txt").read())
vm = run_virtual_machine(input_value=1)
print(f"Part 1: {first(vm.outputs)}")  # 2453265701
vm = run_virtual_machine(input_value=2)
print(f"Part 2: {first(vm.outputs)}")  # 80805

Part 1: 2453265701
Part 2: 80805


In [156]:
# Day 10: Monitoring Station
from math import sqrt


@dataclass(frozen=True)
class Point:
    x: int
    y: int


def on_line(start, end, point):
    """True if point is exactly on the line between start and end.

    Implementation from https://stackoverflow.com/questions/328107/how-can-you-determine-a-point-is-between-two-other-points-on-a-line-segment"""
    crossproduct = (point.y - start.y) * (end.x - start.x) - (point.x - start.x) * (
        end.y - start.y
    )
    if abs(crossproduct) > 0.000001:
        return False

    dotproduct = (point.x - start.x) * (end.x - start.x) + (point.y - start.y) * (
        end.y - start.y
    )
    if dotproduct < 0:
        return False

    squared_length = (end.x - start.x) ** 2 + (end.y - start.y) ** 2
    if dotproduct > squared_length:
        return False
    return True


def direct_line_of_sight(a, b, points):
    points = points - {a, b}
    return not any(on_line(a, b, point) for point in points)


def can_see_count(asteroids):
    result = Counter()
    for a, b in combinations(asteroids, r=2):
        if direct_line_of_sight(a, b, asteroids):
            result.update({a, b})
    return result


def angle(start, point):
    "Angle between start and point. Min value straight up."
    x = point.x - start.x
    y = point.y - start.y
    result = atan2(y, x)
    if -pi < result < -pi / 2:
        # Top left quadrant
        return result + 2 * pi
    else:
        return result


def can_see(point, asteroids):
    return [a for a in asteroids if direct_line_of_sight(point, a, asteroids)]


lines = open("2019/10.txt").read().splitlines()
asteroids = {
    Point(x, y)
    for y, line in enumerate(lines)
    for x, pos in enumerate(line)
    if pos == "#"
}

c = can_see_count(asteroids)
print(f"Part 1: {max(c.values())}")  # 292 (10s execution time)

# Part 2
start = c.most_common(1)[0][0]
seen = can_see(start, asteroids)
seen.sort(key=lambda x: angle(start, x))
target = seen[199]
print(f"Part 2: {target.x * 100 + target.y}")  # 317

Part 1: 292
Part 2: 317


In [275]:
# Day 11: Space Police
# Day 9: Sensor Boost
@dataclass
class VirtualMachine:
    inputs: List[int]
    outputs: List[int]
    memory: DefaultDict[int, int]
    ip: int = 0
    relative_base: int = 0
    running: bool = True


def address(offset, modes):
    POSITION = "0"
    IMMEDIATE = "1"
    RELATIVE = "2"
    if modes[-offset] == POSITION:
        return memory[ip + offset]
    elif modes[-offset] == IMMEDIATE:
        return ip + offset
    elif modes[-offset] == RELATIVE:
        return memory[ip + offset] + relative_base
    else:
        raise ValueError("Invalid mode", modes[-offset], offset, modes)


def add(modes):
    global ip
    memory[address(3, modes)] = memory[address(1, modes)] + memory[address(2, modes)]
    ip += 4


def mul(modes):
    global ip
    memory[address(3, modes)] = memory[address(1, modes)] * memory[address(2, modes)]
    ip += 4


def read_input(modes):
    global ip, inputs
    if inputs:
        memory[address(1, modes)] = inputs.pop(0)
    else:
        raise RuntimeError("No values in inputs")
    ip += 2


def output(modes):
    global ip
    mem_address = address(1, modes)
    ip += 2
    return memory[mem_address]


def jump_if_true(modes):
    global ip
    if memory[address(1, modes)]:
        ip = memory[address(2, modes)]
    else:
        ip += 3


def jump_if_false(modes):
    global ip
    if not memory[address(1, modes)]:
        ip = memory[address(2, modes)]
    else:
        ip += 3


def less_than(modes):
    global ip
    if memory[address(1, modes)] < memory[address(2, modes)]:
        memory[address(3, modes)] = 1
    else:
        memory[address(3, modes)] = 0
    ip += 4


def equals(modes):
    global ip
    if memory[address(1, modes)] == memory[address(2, modes)]:
        memory[address(3, modes)] = 1
    else:
        memory[address(3, modes)] = 0
    ip += 4


def relative_base_offset(modes):
    global ip, relative_base
    relative_base += memory[address(1, modes)]
    ip += 2


def run_program(vm):
    global ip, memory, inputs, relative_base
    instructions = {
        1: add,
        2: mul,
        3: read_input,
        4: output,
        5: jump_if_true,
        6: jump_if_false,
        7: less_than,
        8: equals,
        9: relative_base_offset,
    }
    memory = vm.memory.copy()
    inputs = vm.inputs.copy()
    ip = vm.ip
    relative_base = vm.relative_base
    opcode = int(str(memory[ip])[-2:])  # Last two digits of instruction field
    while opcode != 99:
        modes = "000" + str(memory[ip])[:-2]  # Everything but last two digits, padded
        if instructions[opcode] == read_input and not inputs:
            # Waiting for inputs. Store program state and let the next amplifier run
            vm.memory = memory.copy()
            vm.inputs = inputs.copy()
            vm.ip = ip
            vm.relative_base = relative_base
            return
        result = instructions[opcode](modes)  # Execute instruction
        if instructions[opcode] == output:
            vm.outputs.append(result)
        opcode = int(str(memory[ip])[-2:])  # Next opcode
    vm.running = False


def create_program(text):
    program = defaultdict(int)
    for pos, x in enumerate(text.split(",")):
        program[pos] = int(x)
    return program


def draw(program, start_color):
    panels = defaultdict(int)
    dirs = [
        (-1, 0),
        (0, 1),
        (1, 0),
        (0, -1),
    ]  # dx, dy for up, right, down, left
    x = y = dir_index = 0
    vm = VirtualMachine([start_color], [], program)
    while vm.running:
        run_program(vm)
        # Paint (first output)
        panels[(x, y)] = vm.outputs.pop(0)
        # Turn (second output)
        turn_right = vm.outputs.pop(0)
        if turn_right == 1:
            dir_index = (dir_index + 1) % 4
        else:
            dir_index = (dir_index - 1) % 4
        # Move
        dx, dy = dirs[dir_index]
        x += dx
        y += dy
        vm.inputs = [panels[(x, y)]]
    return panels


program = create_program(open("2019/11.txt").read())
BLACK, WHITE = 0, 1

print(f"Part 1: {len(draw(program, BLACK))}")  # 1876
print("Part 2:")
panels = draw(program, WHITE)
xmax, ymax = max(panels)
for x in range(xmax + 1):
    for y in range(ymax + 1):
        print("⬛" if panels[(x, y)] else "⬜️", end="")
    print()

Part 1: 1876
Part 2:
⬜️⬜️⬛⬛⬜️⬜️⬜️⬛⬛⬜️⬜️⬛⬛⬛⬜️⬜️⬜️⬜️⬛⬛⬜️⬜️⬛⬛⬜️⬜️⬜️⬛⬛⬜️⬜️⬜️⬛⬛⬜️⬜️⬛⬜️⬜️⬜️⬜️
⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬜️⬜️
⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬜️⬜️
⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬛⬛⬜️⬛⬛⬛⬜️⬜️⬜️⬜️⬜️⬛⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬛⬛⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬜️⬜️
⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬜️⬜️
⬜️⬜️⬛⬛⬜️⬜️⬜️⬛⬛⬛⬜️⬛⬜️⬜️⬜️⬜️⬜️⬛⬛⬜️⬜️⬜️⬛⬛⬜️⬜️⬜️⬛⬛⬛⬜️⬜️⬛⬛⬜️⬜️⬛⬛⬛⬛⬜️


In [345]:
# Day 12: The N-Body Problem
@dataclass
class Moon:
    position: Tuple[int, int, int]
    velocity: List[int] = field(default_factory=lambda: [0, 0, 0])


def apply_gravity(a, b):
    def adjust_velocity(pos):
        if a.position[pos] < b.position[pos]:
            a.velocity[pos] += 1
            b.velocity[pos] -= 1
        elif a.position[pos] > b.position[pos]:
            a.velocity[pos] -= 1
            b.velocity[pos] += 1

    for i in range(3):
        adjust_velocity(i)


def apply_velocity(moon):
    new_position = tuple(moon.position[i] + moon.velocity[i] for i in range(3))
    moon.position = new_position


def time_step(moons):
    for a, b in combinations(moons, r=2):
        apply_gravity(a, b)
    for moon in moons:
        apply_velocity(moon)


def potential_energy(moon):
    return sum(abs(coord) for coord in moon.position)


def kinetec_energy(moon):
    return sum(abs(coord) for coord in moon.velocity)


def total_energy(moon):
    return potential_energy(moon) * kinetec_energy(moon)


lines = open("2019/12.txt").read().splitlines()
moons = [Moon(tuple(int(x) for x in re.findall(r"-?\d+", line))) for line in lines]
for _ in range(1000):
    time_step(moons)
print(f"Part 1: {sum(total_energy(moon) for moon in moons)}")

Part 1: 183


In [420]:
moons = [Moon((-1, 0, 2)), Moon((2, -10, -7)), Moon((4, -8, 8)), Moon((3, 5, -1))]
original = [Moon((-1, 0, 2)), Moon((2, -10, -7)), Moon((4, -8, 8)), Moon((3, 5, -1))]
lines = """<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>""".splitlines()
moons = [Moon(tuple(int(x) for x in re.findall(r"-?\d+", line))) for line in lines]
original = [Moon(tuple(int(x) for x in re.findall(r"-?\d+", line))) for line in lines]

steps_since_last_repeated_state = defaultdict(lambda: set())
last_repeated_step = [0] * len(moons)
for step in range(1, 10_000_000):
    time_step(moons)
    for i, _ in enumerate(moons):
        if moons[i] == original[i]:
            steps_since_last_repeated_state[i].add((step - last_repeated_step[i]))
            last_repeated_step[i] = step

In [421]:
steps_since_last_repeated_state

defaultdict(<function __main__.<lambda>()>, {})